### Descripción del proyecto

Film Junky Union, una nueva comunidad vanguardista para los aficionados de las películas clásicas, está desarrollando un sistema para filtrar y categorizar reseñas de películas. Tu objetivo es entrenar un modelo para detectar las críticas negativas de forma automática. Para lograrlo, utilizarás un conjunto de datos de reseñas de películas de IMDB con etiquetado para construir un modelo que clasifique las reseñas como positivas y negativas. Este deberá alcanzar un valor F1 de al menos 0.85.

#### Descripción de los datos
Los datos se almacenan en el archivo imdb_reviews.tsv.

Los datos fueron proporcionados por Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, y Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. La Reunión Anual 49 de la Asociación de Lingüística Computacional (ACL 2011).

Aquí se describen los campos seleccionados:

review: el texto de la reseña
pos: el objetivo, '0' para negativo y '1' para positivo
ds_part: 'entrenamiento'/'prueba' para la parte de entrenamiento/prueba del conjunto de datos, respectivamente
Hay otros campos en el conjunto de datos, puedes explorarlos si lo deseas.

#### Carga los datos.

In [1]:
# Librerías principales
import numpy as np  # Operaciones matemáticas y manejo de arrays
import pandas as pd  # Manejo y análisis de datos
import re  # Manejo de expresiones regulares

# Scikit-learn: procesamiento de datos, modelos y métricas
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [2]:
# Carga de datos
raw_reviews = pd.read_csv('../raw/imdb_reviews.tsv', sep='\t')
raw_reviews.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47331 entries, 0 to 47330
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           47331 non-null  object 
 1   title_type       47331 non-null  object 
 2   primary_title    47331 non-null  object 
 3   original_title   47331 non-null  object 
 4   start_year       47331 non-null  int64  
 5   end_year         47331 non-null  object 
 6   runtime_minutes  47331 non-null  object 
 7   is_adult         47331 non-null  int64  
 8   genres           47331 non-null  object 
 9   average_rating   47329 non-null  float64
 10  votes            47329 non-null  float64
 11  review           47331 non-null  object 
 12  rating           47331 non-null  int64  
 13  sp               47331 non-null  object 
 14  pos              47331 non-null  int64  
 15  ds_part          47331 non-null  object 
 16  idx              47331 non-null  int64  
dtypes: float64(2

#### Preprocesamiento de datos

In [3]:
# Revisamos si hay duplicados
raw_reviews.duplicated().value_counts()

False    47331
Name: count, dtype: int64

In [18]:
def null_summary(df):
    """
    Calcula el número de valores nulos y el porcentaje de valores nulos para cada columna del DataFrame.
    
    Args:
    df (pd.DataFrame): El DataFrame sobre el que calcular los valores nulos.
    
    Returns:
    pd.DataFrame: Un DataFrame con el número y el porcentaje de valores nulos por columna.
    """
    # Calculamos el número de valores nulos por columna
    not_null_count = df.notnull().sum()
    
    # Calculamos el número de valores nulos por columna
    null_count = df.isnull().sum()
    
    # Calculamos el porcentaje de valores nulos por columna
    null_percentage = (null_count / len(df)) * 100
    
    # Creamos un DataFrame con los resultados
    null_data = pd.DataFrame({
        'Not Null Count': not_null_count,
        'Null Count': null_count,
        'Null Percentage': null_percentage
    })
    
    # Ordenamos por la cantidad de valores nulos de mayor a menor
    null_data = null_data.sort_values(by='Null Count', ascending=False)
    
    print(null_data)

In [19]:
null_summary(raw_reviews)

                 Not Null Count  Null Count  Null Percentage
votes                     47329           2         0.004226
average_rating            47329           2         0.004226
tconst                    47331           0         0.000000
title_type                47331           0         0.000000
primary_title             47331           0         0.000000
end_year                  47331           0         0.000000
runtime_minutes           47331           0         0.000000
original_title            47331           0         0.000000
start_year                47331           0         0.000000
genres                    47331           0         0.000000
is_adult                  47331           0         0.000000
review                    47331           0         0.000000
rating                    47331           0         0.000000
sp                        47331           0         0.000000
pos                       47331           0         0.000000
ds_part                 

In [20]:
# Revisamos la columna que contiene los objetivos o target, verificamos el balance
raw_reviews['pos'].value_counts(normalize=True) * 100 

pos
0    50.104583
1    49.895417
Name: proportion, dtype: float64

In [22]:
# Para el presente analisis solo requerimos de estas 3 columnas
df_reviews = raw_reviews[['review','pos','ds_part']] 
df_reviews


,review,pos,ds_part
0,The pakage implies that Warren Beatty and Gold...,0,train
1,How the hell did they get this made?! Presenti...,0,train
2,There is no real story the film seems more lik...,0,test
3,Um .... a serious film about troubled teens in...,1,test
4,I'm totally agree with GarryJohal from Singapo...,1,test
...,...,...,...
47326,This is another of my favorite Columbos. It sp...,1,test
47327,Talk about being boring! I got this expecting ...,0,test
47328,"I never thought I'd say this about a biopic, b...",1,test
47329,Spirit and Chaos is an artistic biopic of Miya...,1,test


### Comentarios
- No se encontraron valores duplicados en el conjunto de datos, lo que indica que no es necesario realizar acciones adicionales para tratar duplicados.
- Identificamos valores NaN en dos columnas, pero estas no son relevantes para el análisis ni para el objetivo principal del proyecto, por lo que no afectarán los resultados.
- La columna pos presenta una distribución equilibrada de sus categorías, lo que confirma que no hay un problema de desequilibrio de clases en los datos.
- El conjunto de datos contiene la información esencial requerida para el análisis, lo que permite proceder con confianza hacia las siguientes etapas del proyecto.

#### Realizamos el Analisis

In [25]:
# Eliminar caracteres especiales de las reseñas y convertir a minúsculas
def clean_text(text):
    """
    Limpia un texto eliminando caracteres especiales y convirtiéndolo a minúsculas.
    
    Parámetros:
    - text (str): Texto que será limpiado.
    
    Retorna:
    - str: Texto limpio.
    """
    return re.sub(r'[^\w\s]', '', text.lower())

In [26]:
# Limpiamos las reseñas y creamos una nueva columna 'clean_review' usando assign
df_reviews = df_reviews.assign(clean_review=df_reviews['review'].apply(clean_text))

# Dividimos los datos en conjuntos de entrenamiento y prueba
features = df_reviews['clean_review']
target = df_reviews['pos']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [27]:
# Convertir texto en vectores de palabras, usamos CountVectorizer por ser el modelo de procesamiento de lenguaje mas simple.
vectorizer = CountVectorizer(stop_words='english')
features_train_vec = vectorizer.fit_transform(features_train)
features_test_vec = vectorizer.transform(features_test)

#### Entrena al menos tres modelos diferentes para el conjunto de datos de entrenamiento.
Probaremos los modelos para el conjunto de datos de prueba.

In [28]:
# Entrenamos 3 modelos para compararlos.

# Modelo de Regresión Logística
log_model = LogisticRegression(C=0.01, solver='liblinear', random_state=42)
log_model.fit(features_train_vec, target_train)
log_pred = log_model.predict(features_test_vec)
log_f1 = f1_score(target_test, log_pred)

# Modelo de Árbol de Decisión
tree_model = DecisionTreeClassifier(max_depth=15, random_state=42)
tree_model.fit(features_train_vec, target_train)
tree_pred = tree_model.predict(features_test_vec)
tree_f1 = f1_score(target_test, tree_pred)

# Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20)
rf_model.fit(features_train_vec, target_train)
rf_pred = rf_model.predict(features_test_vec)
rf_f1 = f1_score(target_test, rf_pred)

# Mostrar los F1 Scores
print(f'Logistic Regression F1: {log_f1}')
print(f'Decision Tree F1: {tree_f1}')
print(f'Random Forest F1: {rf_f1}')

Logistic Regression F1: 0.8827050299275438
Decision Tree F1: 0.7666506947771922
Random Forest F1: 0.8555818987733224


#### Escribiremos algunas reseñas y vamos a clasificarlas con todos los modelos para ver como se comportan

In [ ]:
# Crear nuevas reseñas
new_reviews = [
               "This movie was excellent and very enjoyable!",  # review positivo
               "I hated this film, it was the worst experience I've had.", # review negativo
               "The movie had its moments, but overall, I felt like something was missing" # review ambiguo (negativo)
               ]

# Preprocesar y convertir las nuevas reseñas
new_reviews_cleaned = [clean_text(review) for review in new_reviews]
new_reviews_vec = vectorizer.transform(new_reviews_cleaned)

# Predecir con los modelos entrenados
log_pred_new = log_model.predict(new_reviews_vec)
tree_pred_new = tree_model.predict(new_reviews_vec)
rf_pred_new = rf_model.predict(new_reviews_vec)

# Mostrar predicciones
print("Logistic Regression Predictions:", log_pred_new)
print("Decision Tree Predictions:", tree_pred_new)
print("Random Forest Predictions:", rf_pred_new)


Logistic Regression Predictions: [1 0 0]
Decision Tree Predictions: [1 0 1]
Random Forest Predictions: [1 1 1]


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 14 stored elements and shape (3, 142456)>

#### Comparaciones

- Los resultados muestran que Regresión Logística obtuvo el mejor desempeño con una puntuación de 0.88, seguida de RandomForestClassifier con 0.85 y finalmente DecisionTreeClassifier con 0.77.
- La Regresión Logística destaca por su simplicidad y es especialmente efectiva cuando los datos no presentan relaciones complejas, lo que explica su buen desempeño en este caso.
- El modelo RandomForestClassifier demostró ser más robusto frente al sobreajuste en comparación con DecisionTreeClassifier, capturando mejor las relaciones presentes en los datos gracias a su enfoque basado en múltiples árboles.
- Aunque es posible optimizar los hiperparámetros de los modelos, los resultados actuales son suficientes para realizar un análisis preliminar y tomar decisiones fundamentadas.

In [42]:
# Comparacion adicional con TF-IDF
# Definir el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, min_df=10)

# Convertir las reseñas de entrenamiento y prueba en vectores TF-IDF
features_train_tfidf = tfidf_vectorizer.fit_transform(features_train)
features_test_tfidf = tfidf_vectorizer.transform(features_test)

# Modelo con tf-idf
log_model_tfidf = LogisticRegression()
log_model_tfidf.fit(features_train_tfidf, target_train)
log_pred_tfidf = log_model_tfidf.predict(features_test_tfidf)
log_f1_tfidf = f1_score(target_test, log_pred_tfidf)

print(f'TF-IDF Logistic Regression F1: {log_f1_tfidf}')

TF-IDF Logistic Regression F1: 0.8955974842767296


In [ ]:
# Probamos las reseñas anteriormente usadas con los modelos anteriores.
#new_reviews_cleaned   # Reseñas procesadas anterioremente.

features_test_tfidf = tfidf_vectorizer.transform(new_reviews_cleaned)

# Predecir con los modelos entrenados
log_pred_tfidf_new = log_model_tfidf.predict(features_test_tfidf)

# Mostrar predicciones
print("TF-IDF Logistic Regression Predictions:", log_pred_tfidf_new)

TF-IDF Logistic Regression Predictions: [1 0 0]


#### Conclusiones Finales

- Los proyectos de clasificación con Regresión Logística en lenguaje natural destacan por su simplicidad y efectividad. La parte más desafiante del proceso radica en la preparación de los datos de texto, específicamente en las etapas de tokenización y lemmatización, aunque hoy en día existen herramientas y modelos preentrenados que simplifican estas tareas.
- Las técnicas de procesamiento de texto varían desde las más simples, como CountVectorizer y TF-IDF, hasta modelos más avanzados como BERT. Este último sobresale por su capacidad para entender el contexto semántico de los textos, aunque requiere mayor tiempo y recursos computacionales debido a su complejidad.
En este proyecto, logramos identificar que la Regresión Logística es el algoritmo más adecuado para problemas de esta naturaleza, destacando su balance entre simplicidad y desempeño.
- Usando TF-IDF como técnica de vectorización, alcanzamos un F1-Score de 0.89 con Regresión Logística, lo que representa una mejora significativa frente a otras técnicas más simples.
- Aunque intentamos implementar BERT, el procesamiento en lotes de 100 reseñas resultó extremadamente lento (más de 40 minutos sin resultados finales), lo que refuerza la necesidad de recursos especializados para su uso efectivo.
- Notamos que con TF-IDF, el F1-Score se incrementó hasta un 89.6%, mostrando su capacidad para capturar características relevantes del texto.
- Es importante destacar que los modelos con F1-Scores de 88% o superiores lograron clasificar correctamente todas las reseñas evaluadas, lo que refuerza su confiabilidad para este tipo de tareas.
